In [ ]:
import pandas as pd 
import pickle
import datetime
import re
import networkx as nx
import numpy as np

# Atrial fibrillation (AF) patients on warfarin

In [ ]:
with open('/projects/ROCKET AF/1A/warfarin dataextraction/data/first_mention_warfarin.pickle', 'rb') as f:
    warfarin = pickle.load(f)

In [ ]:
len(warfarin)

In [ ]:
df = pd.DataFrame(list(warfarin.items()), columns=['client_idcode', 'first_mention_warfarin']) # This code converts a dictionary (warfarin) into a Pandas DataFrame with two columns named 'client_idcode' and 'first_mention_warfarin'
df.shape

In [ ]:
df.head()

In [ ]:
ids = set(df['client_idcode'])
len(ids)

# AF patients on warfarin with a left ventricular ejection fraction (LVEF) <35%

In [ ]:
lvef = pd.read_csv('/projects/data/GS/HF/from_jack/FULL_TEXT_cardiac_echo_report_v2_with_vars.csv')

In [ ]:
lvef.shape

In [ ]:
lvef.head()

In [ ]:
lvef['document_datecreated'].isna().sum()

In [ ]:
lvef = lvef[['client_idcode', 'document_datecreated', 'ef_value']]

In [ ]:
lvef.head()

In [ ]:
lvef.shape

In [ ]:
lvef.dropna(subset=['ef_value'], inplace = True)

In [ ]:
lvef.shape

In [ ]:
df = pd.merge(lvef, df, on='client_idcode', how='right') 

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df['ef_value'].isna().sum()

In [ ]:
df.dropna(subset=['ef_value'], inplace = True)

In [ ]:
df.shape

In [ ]:
df['client_idcode'].nunique()

In [ ]:
2340 + 999

In [ ]:
df['document_datecreated'] = pd.to_datetime(df['document_datecreated'], utc=True).dt.date 

In [ ]:
df.head()

In [ ]:
%%time 
df['difference'] = pd.to_datetime(df['first_mention_warfarin']) - pd.to_datetime(df['document_datecreated'])
min_time = datetime.timedelta(days=1) 
max_time = datetime.timedelta(days=183)
t1 = df['difference'] >= min_time 
t2 = df['difference'] <= max_time
in_window = t1 & t2 
df['in_window'] = in_window

In [ ]:
df.head()

In [ ]:
df = df.loc[df['in_window'] == True]
df['client_idcode'].nunique()

In [ ]:
df = df[df['ef_value'] <= 35]

In [ ]:
df['client_idcode'].nunique()

In [ ]:
lvef = set(df['client_idcode'])
len(lvef)

In [ ]:
with open('/projects/ROCKET AF/1A/warfarin dataextraction/data/lvef.pickle', 'wb') as f:
    pickle.dump(lvef,f)

# AF patients on warfarin with heart failure (HF)

In [ ]:
def graph_from_onto(onto):
    G = nx.DiGraph()
    cl_edges = []
    for s, ts in onto.items():
        for t in ts:
            cl_edges.append((s.replace('S-', ''), t.replace('S-', '')))
    G.add_edges_from(cl_edges)
    return G

def load_onto(o):
    with open(f'/projects/data/GS/{o}', 'rb') as f:
        onto = pickle.load(f)
    G = graph_from_onto(onto)
    return G

def expand_codes(onto, codes):
    expanded = {}
    for name, top_codes in codes.items():
        expanded[name] = set()
        for c in top_codes:
            expanded[name].add(c)
            if c in onto:
                expanded[name].update(nx.ancestors(onto, c))
            else:
                print("NOT FOUND:", c, name)
        print(name, len(top_codes), len(expanded[name]))
    return expanded

In [ ]:
onto = load_onto('isa_rela_ch2pt_202009.pickle')

In [ ]:
hf = { 'HF' : ['84114007']}
hf = expand_codes(onto,hf)

In [ ]:
%%time 
with open('/projects/data/GS/pt2cui_pos_dates.pickle', 'rb') as f:
    pt2cui_pos_dates = pickle.load(f)

In [ ]:
def dates_ex(pt_data, codes):
    dates = None
    for x in codes:
        if x in pt_data and len(pt_data[x])>=2:
            if dates == None:
                dates = pt_data[x]
            else: 
                dates = dates.union(pt_data[x])
    return dates

rows = []
for pt in ids:
    pt_data = pt2cui_pos_dates.get(pt, {})
    row = {'client_idcode': pt, 'first_mention_warfarin': warfarin[pt]} 
    for concept, codes in hf.items():
        dates = dates_ex(pt_data, codes)
        if dates == None:
            row[f"{concept}_date"] = np.nan
        else:
            row[f"{concept}_date"] = dates
    rows.append(row)

In [ ]:
df = pd.DataFrame(rows)
df.shape

In [ ]:
df.head()

In [ ]:
df.dropna(subset=['HF_date'], inplace = True)

In [ ]:
df.head()

In [ ]:
if not row.empty:
    # Access the set within the 'Set_Column' and print it
    specific_set = row.iloc[0]['HF_date']
    print(f"Set for ID {specific_client_idcode}: {specific_set}")
else:
    print(f"No matching ID found for {specific_client_idcode}")

In [ ]:
df['client_idcode'].nunique()

In [ ]:
%%time
df = df[['client_idcode']].join((df[i].explode() for i in df.iloc[:,1:]))
df.shape

In [ ]:
df.head(6)

In [ ]:
df['HF_date'] = pd.to_datetime(df['HF_date'], utc=True).dt.date 

In [ ]:
df.head()

In [ ]:
%%time 
df['difference'] = pd.to_datetime(df['first_mention_warfarin']) - pd.to_datetime(df['HF_date'])
min_time = datetime.timedelta(days=1) 
max_time = datetime.timedelta(days=183)
t1 = df['difference'] >= min_time 
t2 = df['difference'] <= max_time
in_window = t1 & t2 
df['in_window'] = in_window

In [ ]:
df.head()

In [ ]:
df = df.loc[df['in_window'] == True]
df['client_idcode'].nunique()

In [ ]:
hf = set(df['client_idcode'])
len(hf)

In [ ]:
with open('/projects/ROCKET AF/1A/warfarin dataextraction/data/hf.pickle', 'wb') as f:
    pickle.dump(hf,f)